In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing
import time
import seaborn as sns
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [ ]:
data = pd.read_csv('weatherAUS.csv')

In [ ]:
# removing features withg many null values
data.drop(['Evaporation','Sunshine','Cloud9am','Cloud3pm'],axis=1,inplace=True)

In [ ]:
#replace the nulls with mean values of each feature
data['MinTemp'].fillna(data['MinTemp'].mean(),inplace=True)
data['MaxTemp'].fillna(data['MaxTemp'].mean(),inplace=True)
data['Rainfall'].fillna(data['Rainfall'].mean(),inplace=True)
data['WindGustSpeed'].fillna(data['WindGustSpeed'].mean(),inplace=True)
data['WindSpeed9am'].fillna(data['WindSpeed9am'].mean(),inplace=True)
data['WindSpeed3pm'].fillna(data['WindSpeed3pm'].mean(),inplace=True)
data['Humidity9am'].fillna(data['Humidity9am'].mean(),inplace=True)
data['Humidity3pm'].fillna(data['Humidity3pm'].mean(),inplace=True)
data['Pressure9am'].fillna(data['Pressure9am'].mean(),inplace=True)
data['Pressure3pm'].fillna(data['Pressure3pm'].mean(),inplace=True)
data['Temp9am'].fillna(data['Temp9am'].mean(),inplace=True)
data['Temp3pm'].fillna(data['Temp3pm'].mean(),inplace=True)

In [ ]:
data['WindGustDir'].replace(np.nan,  'W',inplace=True)
data['WindDir9am'].replace(np.nan, 'N',inplace=True)
data['WindDir3pm'].replace(np.nan, 'SE',inplace=True) 
data['RainToday'].replace(np.nan,  'Yes',inplace=True)
data['RainTomorrow'].replace(np.nan, 'Yes',inplace=True)

In [ ]:
# encode 
data['RainToday'] = data['RainToday'].map({'Yes':1,'No':0})
data['RainTomorrow'] = data['RainTomorrow'].map({'Yes':1,'No':0})
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
var_mod = ['Date','Location','WindGustDir','WindDir9am','WindDir3pm']
for i in var_mod:
    data[i] = le.fit_transform(data[i])

In [ ]:
X = data.drop('RainTomorrow',axis=1)
Y = data['RainTomorrow']

In [ ]:

def run_rf(X,Y):
    X_train,X_test, y_train, y_test = train_test_split(X, Y, stratify=Y, test_size=0.33)
    start = time.process_time()
    model = RandomForestClassifier(n_estimators=700)
    model.fit(X_train,y_train)
    print('Run Time is:', time.process_time() - start)
    pred = model.predict(X_test)
    print('\n')
    print('The confusion Matrix is:\n',confusion_matrix(y_test,pred))
    print('\n')
    print('The classification report is:\n',classification_report(y_test,pred))


In [ ]:
# run rf on original data
run_rf(X[:1000],Y[:1000])

.

.

Feature Extraction : PCA

In [ ]:
df1 = data

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
PCA_df = pd.DataFrame(data = X_pca, columns = ['PC1', 'PC2'])
PCA_df = pd.concat([PCA_df, df1['RainTomorrow']], axis = 1)
PCA_df['RainTomorrow'] = LabelEncoder().fit_transform(PCA_df['RainTomorrow'])
PCA_df.head()

In [ ]:
# run rf on PCA 2D
run_rf(X_pca[:1000],Y[:1000])

In [ ]:
figure(num=None, figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')

classes = [1, 0]
colors = ['r', 'b']
for clas, color in zip(classes, colors):
    plt.scatter(PCA_df.loc[PCA_df['RainTomorrow'] == clas, 'PC1'], PCA_df.loc[PCA_df['RainTomorrow'] == clas, 'PC2'], c = color)
    
plt.xlabel('Principal Component 1', fontsize = 12)
plt.ylabel('Principal Component 2', fontsize = 12)
plt.title('2D PCA', fontsize = 15)
plt.legend(['will rain', 'wont rain'])
plt.grid()


In [ ]:
var_ratio = pca.explained_variance_ratio_
cum_var_ratio = np.cumsum(var_ratio)

trace1 = dict(
    type='bar',
    x=['PC %s' %i for i in range(1,5)],
    y=var_ratio,
    name='Each PC'
)

trace2 = dict(
    type='scatter',
    x=['PC %s' %i for i in range(1,5)], 
    y=cum_var_ratio,
    name='Cumulative'
)

data = [trace1, trace2]

layout=dict(
    title='variance Ratio of each principal components in percent',
    yaxis=dict(
        title='variance ratio'
    ),
    annotations=list([
        dict(
            x=1.16,
            y=1.05,
            xref='paper',
            yref='paper',
            showarrow=False,
        )
    ])
)

fig = dict(data=data, layout=layout)
iplot(fig)

In [ ]:
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)
PCA_df = pd.DataFrame(data = X_pca, columns = ['PC1', 'PC2','PC3'])
PCA_df = pd.concat([PCA_df, df1['RainTomorrow']], axis = 1)
PCA_df['RainTomorrow'] = LabelEncoder().fit_transform(PCA_df['RainTomorrow'])
PCA_df.head()

In [ ]:
# run rf on PCA 3D
run_rf(X_pca[:1000],Y[:1000])

In [ ]:
 data = []
for clas, col, name in zip((1, 0), ['red', 'darkblue'], ['will rain', 'wont rain']):
    trace = dict(
        type='scatter3d',
        x= PCA_df.loc[PCA_df['RainTomorrow'] == clas, 'PC1'],
        y= PCA_df.loc[PCA_df['RainTomorrow'] == clas, 'PC2'],
        z= PCA_df.loc[PCA_df['RainTomorrow'] == clas, 'PC3'],
        mode= 'markers',
        name= name
    )
    data.append(trace)
layout = {
    "scene": {
      "xaxis": {
        "title": "PC1", 
        "showline": False
      }, 
      "yaxis": {
        "title": "PC2", 
        "showline": False
      }, 
      "zaxis": {
        "title": "PC3", 
        "showline": False
      }
    }, 
    "title":  'PCA 3D Dimensionality Reduction'
}
fig = dict(data=data, layout=layout)
iplot(fig)

In [ ]:
var_ratio = pca.explained_variance_ratio_
cum_var_ratio = np.cumsum(var_ratio)

trace1 = dict(
    type='bar',
    x=['PC %s' %i for i in range(1,5)],
    y=var_ratio,
    name='Each PC'
)

trace2 = dict(
    type='scatter',
    x=['PC %s' %i for i in range(1,5)], 
    y=cum_var_ratio,
    name='Cumulative'
)

data = [trace1, trace2]

layout=dict(
    title='variance Ratio of each principal components in percent',
    yaxis=dict(
        title='variance ratio'
    ),
    annotations=list([
        dict(
            x=1.16,
            y=1.05,
            xref='paper',
            yref='paper',
            showarrow=False,
        )
    ])
)

fig = dict(data=data, layout=layout)
iplot(fig)

Feature Extraction: ICA

In [ ]:
from sklearn.decomposition import FastICA
ica = FastICA(n_components=3)
X_ica = ica.fit_transform(X)

In [ ]:
# run rf on ICA 3D
run_rf(X_ica[:1000],Y[:1000])

In [ ]:
ICA_df = pd.DataFrame(data = X, columns = ['CA1', 'CA2', 'CA3'])
ICA_df = pd.concat([ICA_df, df1['RainTomorrow']], axis = 1)
ICA_df['RainTomorrow'] = LabelEncoder().fit_transform(ICA_df['RainTomorrow'])
ICA_df.head()

In [ ]:
data = []
for clas, col, name in zip((1, 0), ['red', 'darkblue'], ['will rain', 'wont rain']):

    trace = dict(
        type='scatter3d',
        x= ICA_df.loc[ICA_df['RainTomorrow'] == clas, 'CA1'],
        y= ICA_df.loc[ICA_df['RainTomorrow'] == clas, 'CA2'],
        z= ICA_df.loc[ICA_df['RainTomorrow'] == clas, 'CA3'],
        mode= 'markers',
        name= name
    )
    data.append(trace)

layout = {
    "scene": {
      "xaxis": {
        "title": "CA1", 
        "showline": False
      }, 
      "yaxis": {
        "title": "CA2", 
        "showline": False
      }, 
      "zaxis": {
        "title": "CA3", 
        "showline": False
      }
    }, 
    "title":  'ICA 3D Dimensionality Reduction'
}
# fig = dict(data=data, layout=layout)
# iplot(fig)

Feature Extraction : LDA 

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components=1)

# run an LDA and use it to transform the features
X_lda = lda.fit(X, Y).transform(X)
print('Original number of features:', X.shape[1])
print('Reduced number of features:', X_lda.shape[1])

In [ ]:
# run rf on PCA 3D
run_rf(X_lda[:1000],Y[:1000])

Feature Extraction :  Auto Encoder

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

input_layer = Input(shape=(X.shape[1],))
encoded = Dense(3, activation='relu')(input_layer)
decoded = Dense(X.shape[1], activation='softmax')(encoded)
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

X1, X2, Y1, Y2 = train_test_split(X, X, test_size=0.3, random_state=101)

autoencoder.fit(X1, Y1,
                epochs=10,
                batch_size=300,
                shuffle=True,
                verbose = 30,
                validation_data=(X2, Y2))

encoder = Model(input_layer, encoded)
X_ae = encoder.predict(X)

In [ ]:
# run rf on AE
run_rf(X_ae[:1000],Y[:1000])

In [ ]:
AE_df = pd.DataFrame(data = X, columns = ['AE1', 'AE2', 'AE3'])
AE_df = pd.concat([AE_df, df1['RainTomorrow']], axis = 1)
AE_df['RainTomorrow'] = LabelEncoder().fit_transform(AE_df['RainTomorrow'])
AE_df.head()

In [ ]:
data = []
for clas, col, name in zip((1, 0), ['red', 'darkblue'], ['will rain', 'wont rain']):

    trace = dict(
        type='scatter3d',
        x= AE_df.loc[AE_df['RainTomorrow'] == clas, 'AE1'],
        y= AE_df.loc[AE_df['RainTomorrow'] == clas, 'AE2'],
        z= AE_df.loc[AE_df['RainTomorrow'] == clas, 'AE3'],
        mode= 'markers',
        name= name
    )
    data.append(trace)

layout = {
    "scene": {
      "xaxis": {
        "title": "AE1", 
        "showline": False
      }, 
      "yaxis": {
        "title": "AE2", 
        "showline": False
      }, 
      "zaxis": {
        "title": "AE3", 
        "showline": False
      }
    }, 
    "title": 'AE 3D Dimensionality Reduction'
}
fig = dict(data=data, layout=layout)
iplot(fig)